# Notebook responsável por alguns aspectos de modelagem OneToOne / OneToFew Embedding (usando MongoEngine)

## Bibliotecas importantes

In [ ]:
from mongoengine import *
import pprint
import numpy as np
import pandas as pd
import unidecode

# para geração de dados fake
from faker import Faker


## Classes úteis

In [ ]:
class Endereco (EmbeddedDocument):
    logradouro = StringField(max_length=200, required=True)
    numero = StringField(max_length=10, required=True)
    bairro = StringField(max_length=50, required=True)
    cidade = StringField(max_length=200, required=True)
    uf = StringField(max_length=50, required=True)
    cep = StringField(max_length=10, required=True)
    
class Pessoa (Document):
    cpf = StringField(max_length=20, required=True)
    primeiro_nome = StringField(max_length=200, required=True)
    ultimo_nome = StringField(max_length=400, required=True)
    idade = IntField(min_value=0, max_value=120, required=True)
    data_nascimento = StringField(max_length=20, required=True)
    email = EmailField(max_length=50)
    telefones = ListField(StringField(max_length=50), max_length=4)
    formacoes = ListField(StringField(max_length=50), max_length=4)
    
    # Referência (linking) para um objeto da classe pessoa
    endereco = EmbeddedDocumentField(Endereco)

## Funções uteis

In [ ]:
def criar_endereco():
    endereco = Endereco()
    
    endereco.logradouro = fake.street_name()
    endereco.numero = fake.building_number()
    endereco.bairro = fake.bairro()
    endereco.cidade = fake.city()
    endereco.uf = fake.administrative_unit()
    endereco.cep = fake.postcode()

    return endereco

In [ ]:
def criar_pessoa():
    
    pessoa = Pessoa()
    pessoa.primeiro_nome = fake.first_name()
    pessoa.ultimo_nome = fake.last_name()
    pessoa.idade = np.random.randint(18, 50)
    pessoa.cpf = fake.cpf()
    pessoa.data_nascimento = fake.date()
    pessoa.email = f'{unidecode.unidecode(pessoa.primeiro_nome.lower())}.\
                     {unidecode.unidecode(pessoa.ultimo_nome.lower())}@{fake.free_email_domain()}'
    
    # criando os telefones
    telefones = []
    n = np.random.randint(1, 4)
    for i in range(n):
        telefones.append(fake.msisdn())
    pessoa.telefones=telefones
    
    # criando as formações
    formacoes = []
    n = np.random.randint(1, 4)
    for i in range(n):
        formacoes.append(fake.job())
    pessoa.formacoes=formacoes
    
    pessoa.endereco = criar_endereco()
    
    return pessoa

## Configurações

In [ ]:
pp = pprint.PrettyPrinter(compact=True)
# gerando dados fictícios em português do Brasil
fake = Faker(['pt_BR'])

## Inserção de vários documentos

### Conectando ao MongoDB

In [ ]:
connect('onetoone_embedding')

### Exemplo - Criação de vários documentos compostos

In [ ]:
lista_pessoas = []

for i in range(10):
    lista_pessoas.append(criar_pessoa())
    
pp.pprint(lista_pessoas)

In [ ]:
pp.pprint(lista_pessoas[0].pk)

### Inserindo os documentos no banco de dados

In [ ]:
# inserindo 1 documento apenas
lista_pessoas[0].save()

In [ ]:
# inserindo todos os outros documentos
Pessoa.objects.insert(lista_pessoas[1:])

## Recuperando os documentos inseridos

#### Recuperar todos

In [ ]:
for p in Pessoa.objects:
    pp.pprint(p.id)